In [ ]:
import intake
import pandas as pd
import hvplot.pandas

#### Guess reader from existing code

In [ ]:
url = "s3://mymdtemp/intake_1.csv"

In [ ]:
df = pd.read_csv(url, storage_options={"anon": True}, usecols=[1, 2])

In [ ]:
reader = intake.reader_from_call(_i)

In [ ]:
reader

In [ ]:
reader.kwargs

#### Or guess from the URL alone

In [ ]:
# uses URL alone, but can also match on magic bytes
intake.datatypes.recommend(url)

In [ ]:
data = intake.readers.datatypes.CSV(url, storage_options={"anon": True})

#### "What can read this?"

In [ ]:
data.possible_outputs

In [ ]:
data.possible_readers

In [ ]:
# same reader as original
# reader = data.to_reader("pandas:DataFrame")
reader = intake.readers.readers.PandasCSV(data)

```python
class PandasCSV(Pandas):
    implements = {datatypes.CSV}
    func = "pandas:read_csv"
    url_arg = "filepath_or_buffer"

    def discover(self, **kw):
        kw["nrows"] = 10
        kw.pop("skipfooter", None)
        kw.pop("chunksize", None)
        return self.read(**kw)
```

#### Reader API

In [ ]:
reader.read()

In [ ]:
print(reader.doc())

In [ ]:
# known transforms and what they make
reader.transform

In [ ]:
# but have access to full DataFrame API
dir(reader)

In [ ]:
# or "pd" namespace (useful for some packages)
reader.pd

#### So lets make a catalog and a pipeline using pandas syntax

In [ ]:
cat = intake.entry.Catalog()
cat["tute"] = reader
cat["capitals"] = reader.a.str.capitalize()
cat["inverted"] = reader.sort_values("b", ascending=False)
cat["multi"] = cat.tute.assign(c=cat.capitals)  # <- uses multiple readers

In [ ]:
reader.a.str.capitalize()

In [ ]:
# what gets stored in the catalog entry?
cat.entries["multi"].kwargs

In [ ]:
cat

In [ ]:
cat.tute.read()

In [ ]:
cat.data  # just one data item

#### To and from catalog file, which you can put anywhere

In [ ]:
cat.to_yaml_file("intake_1.yaml")

In [ ]:
# a "shared" one I prepared for everyone
cat = intake.from_yaml_file("s3://mymdtemp/intake_1.yaml", anon=True)

In [ ]:
# yes, you have <tab> completion
cat.tute

In [ ]:
cat.inverted.read()

#### And now you can go about your work; but some convenience functions might still be useful.

In [ ]:
# add arguments to make a reader you can persist
cat.inverted.ToHvPlot(explorer=True).read()

In [ ]:
cat.inverted.ToMatplotlib.read()

In [ ]:
# you can even have Intake guess the whole pipeline
intake.auto_pipeline(data, "PNG", avoid="Geo")

#### But pandas was not the only engine that can work on this data. We can play with the API or make more readers to persist in the catalog.

In [ ]:
data

In [ ]:
data.to_reader("dask").read()

In [ ]:
data.to_reader("ray").read()

In [ ]:
# dask-on-ray!
data.to_reader("dask").DaskToRay.read()